## Radio interferometría y síntesis de imágenes en astronomía - Laboratorio 1
### Vicente Mieres

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta, timezone
import matplotlib.dates as mdates
import juliandate as jd
import math

## Funciones y utilidades

In [13]:
def ecef_to_enu(ecef, array_center, phi=-33.45, lamb=-70.66, rad=True, is_array=True):
  '''
  ecef in meters (xyz)
  array_center: central antenna coords
  phi: latitud
  lamb (lambda): longitude
  rad: True if phi and lambd need to be radians
  '''
  if rad:
      phi, lamb = np.deg2rad(phi), np.deg2rad(lamb)

  cphi, clam = np.cos(phi), np.cos(lamb)
  sphi, slam = np.sin(phi), np.sin(lamb)


  R = np.array([[-slam,        clam,       0],
                [-clam*sphi,   -slam*sphi, cphi],
                [clam*cphi,    slam*cphi,  sphi]])

  dxyz = np.array(ecef) - np.array(array_center)
  
  enu = R @ dxyz.T if is_array else R @ dxyz

  return enu


def enu_to_altaz(enu, rad=True, is_array=True):
  '''
  Transform the enu vector to altitude and azimut
  '''

  if is_array:
    E, N, U = enu[0], enu[1], enu[2]
  else:
    E, N, U = float(enu[0]), float(enu[1]), float(enu[2])

  r = np.hypot(E, N)           
  El = np.arctan2(U, r)
  A = np.arctan2(E, N)
  
  if rad:
      return A % (2*np.pi), El  
  else:
      A_deg = np.degrees(A) % 360.0
      El_deg = np.degrees(El)
      return A_deg, El_deg

def local_sidereal_time(longitude=-70.76, utc=None, single=True):
  """
  Calculates the local sidereal time in Radians.
  """
  if utc is None:
    now = datetime.now(timezone.utc)
  else:
    now = utc

  jd_now = jd.from_gregorian(now.year, now.month, now.day, now.hour, now.minute, now.second)
  T = (jd_now - 2451545.0) / 36525
  theta = 280.46061837 + 360.98564736629 * (jd_now - 2451545) + (0.000387933 * T * T) - (T * T * T / 38710000.0)
  deg = theta % 360 + longitude

  h,m,s = degree_to_time(deg)
  rad = np.deg2rad(deg)
  
  if single: 
    return rad 
  else: 
    return deg, rad, h, m, s

def degree_to_time(theta, is_rad=False):
  """
  Converts degrees to hours, minutes and seconds.
  """
  if is_rad:
    theta = np.rad2deg(theta)

  h = int(theta/ 15)
  m = (int)(((theta / 15) - h) * 60)
  s = ((((theta / 15) - h) * 60) - m) * 60
  return h, m, s

def Rz(theta):
  c, s = np.cos(theta), np.sin(theta)
  return np.array([[ c, -s, 0],
                   [ s, c, 0],
                   [ 0, 0, 1]])

def Ry(theta):
  c, s = np.cos(theta), np.sin(theta)
  return np.array([[ c, 0, s],
  [ 0, 1, 0],
  [-s, 0, c]])

def hor_to_eq(alt, az, lst, phi=-33.44, is_array=True):
  """
  Transforma coordenadas horizontales (Alt, Az) -> ecuatoriales (α, δ).
  Parámetros
  ----------
  alt : Altitud (elevación) en radianes.
  az : Azimut en radianes (medido desde el Norte hacia el Este).
  lat : Latitud del observador en radianes.
  lst : Tiempo sideral local en radianes.
  Retorna
  -------
  alpha : Ascensión recta (α) en radianes, en el rango [0, 2π).
  delta : Declinación (δ) en radianes.
  """
  # --- Vector en sistema horizontal ENU ---
  # ENU = (East, North, Up)
  E = np.sin(az) * np.cos(alt)
  N = np.cos(az) * np.cos(alt)
  U = np.sin(alt)
  
  R = Rz(lst) @ Ry(phi)

  if is_array:
    v_hor = np.stack([E, N, U], axis=-1)
    r_eq = v_hor @ R.T 
    x, y, z = r_eq[:,0], r_eq[:,1], r_eq[:,2]
  else:
    v_hor = np.array([E, N, U])   
    r_eq = R @ v_hor
    x, y, z = r_eq

  # --- Extraer (α, δ) ---  
  delta = np.arcsin(np.clip(z, -1.0, 1.0))     # declinación
  alpha = np.mod(np.arctan2(y, x), 2*np.pi)    # ascensión recta (0..2π)

  return alpha, delta, r_eq

def eq_to_uvw(H, delta, hor_coords, is_array=True):
  '''
  Return the uvw coords of an antenna o antenna array 
  '''
  ch, cd = np.cos(H), np.cos(delta)
  sh, sd = np.sin(H), np.sin(delta)

  R = np.array([[sh,        ch,      0],
                [-sd*ch,   sd*sh,   cd],
                [cd*ch,   -cd*ch,   sd]])
  
  uvw = hor_coords @ R.T if is_array else R @ hor_coords

  return uvw

def read_cfg_to_enu(filename, array_center=None ,phi=-33.44, lamb=-70.76, rad=True):
  '''
  Read file and return antenna config on ENU coords
  '''
  with open(filename, "r") as f:
    lines = f.readlines()

  coordsys = None 
  for line in lines:
        if line.startswith("# coordsys"):
            coordsys = line.split("=")[1].strip()
            break
  
  antennas = []
  for line in lines:
      if line.startswith("#") or not line.strip():
          continue
      parts = line.split()
      x, y, z = map(float, parts[:3])
      antennas.append([x, y, z])
  antennas = np.array(antennas)

  if coordsys == "LOC (local tangent plane)": return antennas.T
  elif coordsys == "XYZ":
     array_center = array_center if array_center is not None else antennas.mean(axis=0)
     enu_antennas = ecef_to_enu(antennas, array_center, phi, lamb, rad)
     return enu_antennas
  else:
    raise ValueError(f"coordsys desconocido: {coordsys}")


In [26]:
center = [5111214.575406, 2001317.289008, -3237315.843578]

enu = read_cfg_to_enu("../antenna_arrays/kat-7.cfg", array_center=center)

alt, az = enu_to_altaz(enu)

lst = local_sidereal_time(single=True)

d, a, l= hor_to_eq(alt,az, lst)


d

array([4.00908465, 3.03509707, 0.35653558, 1.00932241, 5.12185401,
       4.74592253, 0.89726792])

## Parte 1: Simulación de cobertura *uv*